In [ ]:
import pandas as pd
import numpy as np
import os


In [1]:
input_data = pd.read_csv('data.csv')

NameError: name 'pd' is not defined

In [ ]:
input_data = input_data.iloc[:, :68]

In [ ]:
input_data.to_csv('data.csv', index=False)
input_data.replace('FALSE', 0, inplace=True)

In [ ]:
columns_to_select = [
    'Team Number', 'AC1', 'AC2', 'AC3', 
    'AC4', 'AARL', 'AARH', 'AAP', 'AAB', 'AM', 'TC1', 
    'TC2', 'TC3', 'TC4', 'TARL', 'TARH', 'TAP', 'TAB'
]

input_data = input_data[columns_to_select]

In [ ]:
averaged_data = input_data.groupby("Team Number").mean().reset_index()

In [ ]:
# Game data for team 1

game1 = [3294,7864,5903]

game1mask = averaged_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game1
), axis=1)

game1data = averaged_data[game1mask]

# Game data for team 2

game2 = [4741,6318,3100]

game2mask = averaged_data.apply(lambda row: any(
    (row['Team Number'] == team)
    for team in game2
), axis=1)

game2data = averaged_data[game2mask]


In [ ]:
game1data.sum()

Team Number    17061.000000
AC1                1.444444
AC2                0.000000
AC3                0.000000
AC4                0.000000
AARL               0.000000
AARH               0.000000
AAP                0.000000
AAB                0.000000
AM                 2.888889
TC1                0.142857
TC2                0.698413
TC3                0.222222
TC4                6.158730
TARL               0.142857
TARH               0.650794
TAP                0.000000
TAB                0.000000
dtype: float64

In [ ]:
game2data.sum()

Team Number    14159.000000
AC1                0.222222
AC2                0.000000
AC3                0.000000
AC4                0.000000
AARL               0.222222
AARH               0.777778
AAP                0.000000
AAB                0.888889
AM                 2.444444
TC1                0.777778
TC2                1.666667
TC3                3.555556
TC4                2.000000
TARL               2.000000
TARH               1.888889
TAP                0.666667
TAB                5.111111
dtype: float64

In [ ]:
# Coral clipping
# Every high algae takes out 2 L3 scores, and every low algae takes out 2 L3 and 2 L2 scores
l4_def = 12
l3_def = 0
l2_def = 6

def clip(input_data):
    series = input_data

    l3_cap = l3_def + (input_data["AARL"] * 2) + (input_data["AARH"] * 2) + (input_data["TARL"] * 2) + (input_data["TARH"] * 2)
    l2_cap = l2_def + (input_data["AARL"] * 2) + (input_data["TARL"] * 2)

    if (series["TC3"] > l3_cap):
        series["TC3"] = l3_cap - series["AC3"]
    
    if (series["TC2"] > l2_cap):
        series["TC2"] = l2_cap - series["AC2"]
    
    return series

clipped1 = clip(game1data.sum())
clipped2 = clip(game2data.sum())

In [ ]:
dataWeights = pd.Series([
    0, 3, 4, 6, 7, 0, 0, 6, 4, 3, 2, 3, 4, 5, 0, 0, 6, 4
])


if len(clipped1) == len(dataWeights):
    weighted_sum = (clipped1.reset_index(drop=True) * dataWeights).sum()
    print(f"Points for team 1: {weighted_sum}")
else:
    print("The number of weights must match the number of values in the series.")

if len(clipped2) == len(dataWeights):
    weighted_sum = (clipped2.reset_index(drop=True) * dataWeights).sum()
    print(f"Points for team 2: {weighted_sum}")
else:
    print("The number of weights must match the number of values in the series.")

Points for team 1: 47.06349206349206
Points for team 2: 66.77777777777777
